In [32]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, KFold, cross_val_score 
from sklearn.metrics import classification_report, mean_absolute_error
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor 
from sklearn.linear_model import LinearRegression

### <center> Ансамбли моделей

**Ансамблем (Ensemble)**  называется алгоритм, который состоит из ***нескольких*** алгоритмов машинного обучения.

Такой подход часто используется для того, чтобы усилить "положительные качества" отдельно взятых алгоритмов, которые сами по себе могут работать слабо, а вот в группе - ансамбле давать хороший результат. Объединение нескольких "слабых" алгоритмов дает "сильный".

**Виды ансамблей:**

**1. Бэггинг (Bagging)** - объединение ***одинаковых*** моделей, но обученных на ***различных*** случайных выборках из ***одного*** набора данных. 

Бэггинг направлен на **уменьшение разброса (дисперсии)** в данных и зачастую данный прием предстает в виде алгоритма Случайного леса (Random Forest), где слабые модели - это довольно глубокие случайные деревья.

**2. Бустинг (Boosting)** - объединение **одинаковых** моделей, обученных ***последовательно***, причем последующая модель должна исправлять ошибки предыдущей. 

Бустинг направлен скорее на **уменьшение смещения в данных**, чем на снижение разброса в них. Поэтому в качестве базовых алгоритмов могут браться модели с достаточно высоким смещением, например, неглубокие случайные деревья. Пример алгоритма - Градиентный бустинг (Gradient Boosting) и AdaBoost. Варианты реализации градиентоного бустинга алгоритмов: **XGBoost, LightGBM, CatBoost**

**3. Стекинг (Stacking)** - объединение ***разнородных*** "слабых" моделей в итоговую "сильную" мета-модель или модель второго уровня.

<a href='https://habr.com/ru/post/561732/' > Короткая статья про ансамбли </a>

<a href='https://ml-handbook.ru/chapters/grad_boost/intro' > Статья про бустинг от ШАД Яндекса </a>

<img src='https://github.com/MalikaL17/course_materials/blob/main/img/bagging.jpg?raw=true'>

### <center> Случайных лес (Random Forest)

**RF (random forest)** — это множество решающих деревьев. В задаче регрессии их ответы ***усредняются***, в задаче классификации принимается решение ***голосованием по большинству***.

**Все деревья строятся независимо по следующей схеме:**

1. Выбирается подвыборка обучающей выборки размера samplesize (с возвращением, т.е. **бустрап-выборка**) – по ней строится дерево (для каждого дерева — своя подвыборка).

2. Для построения каждого расщепления в дереве просматриваем max_features случайных признаков (для каждого нового расщепления — свои случайные признаки).

3. Выбираем наилучшие признак и расщепление по нему (по заранее заданному критерию). Дерево строится, как правило, до исчерпания выборки (пока в листьях не останутся представители только одного класса), но в современных реализациях есть параметры, которые ограничивают высоту дерева, число объектов в листьях и число объектов в подвыборке, при котором проводится расщепление.


**Бутстрап (Bootstrap)** - метод генерации выборок из исходной выборки. Из исходной выборки случайным обрзом "достается" N значенией с повторением (т.е. одно и то же зачение может попасть в выборку несколь раз). Такой метод помогает находить различные стат. оценки, когда выборка маленькая, а так же служит для сэмплирования данных в машинном обучении.

Ниже пример выбора обучающих данных из исходного набра для случайного леса.

<img src="https://github.com/MalikaL17/course_materials/blob/main/img/random_forest.PNG?raw=true">

**Важные параметры:**
    
- **n_estimators** - количество деревьев в ансамбле
- **max_features** - количество случайных признаков в для сэмплинга. По умолчанию - корень из числа признаков
- **bootstrap** - сэпмлировать методом бутстрпапа. По умоланию - true, иначе - использует всю выборку для обучения всего дерева
- **max_samples** - количество наблюдений для сэмплирования. По умолчанию используем весь поднабор

Остальные параметра - такие же, как у дерева: **min_sample_leaf**, **max_depth** ...
Не забывайте фиксировать **random_state** для воспроизводимости результатов!

In [43]:
data = pd.read_csv('titanic.csv')
df = data.copy()
df['Sex'] = df['Sex'].map({'female':1, 'male':0})
df['Embarked'] = df['Embarked'].map({'S':0, 'C':1, 'Q':2})
df['Age'] = df['Age'].fillna(df['Age'].mean())
df['Embarked'] = df['Embarked'].fillna(df['Embarked'].mode()[0])
df['Relatives'] = df['SibSp'] + df['Parch'] 
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Relatives
0,1,0,3,"Braund, Mr. Owen Harris",0,22.0,1,0,A/5 21171,7.2500,NaN,0.0,1
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",1,38.0,1,0,PC 17599,71.2833,C85,1.0,1
2,3,1,3,"Heikkinen, Miss. Laina",1,26.0,0,0,STON/O2. 3101282,7.9250,NaN,0.0,0
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",1,35.0,1,0,113803,53.1000,C123,0.0,1
4,5,0,3,"Allen, Mr. William Henry",0,35.0,0,0,373450,8.0500,NaN,0.0,0


**Дерево решений**

In [22]:
X = df[['Pclass', 'Sex', 'Age', 'Fare', 'Embarked', 'Relatives']]
y = df['Survived']

kf = KFold(n_splits=5, shuffle=True, random_state=42)

tree = DecisionTreeClassifier(random_state=17) #

scores = cross_val_score(tree, X, y, cv=kf, scoring='f1')
print("Значения точности при перекрестной проверке: \n{}".format(scores.mean()))

Значения точности при перекрестной проверке: 
0.7311793489592352


**Случайный лес**

In [23]:
from sklearn.ensemble import RandomForestClassifier

In [45]:
X = df[['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Embarked']]
y = df['Survived']

kf = KFold(n_splits=5, shuffle=True, random_state=42)

rf = RandomForestClassifier(n_estimators=200, random_state=17)

scores = cross_val_score(rf, X, y, cv=kf, scoring='f1') 
print("Значения точности при перекрестной проверке: \n{}".format(scores.mean()))

Значения точности при перекрестной проверке: 
0.7531121013523187


### <center> Градиентный бустинг (Gradient Boosting)

При использовании линейной регрессии делается предположение о нормальности распределения остатков вокруг нуля. Пример остатков для хорошей модели на рисунке ниже.
<img src='https://github.com/MalikaL17/course_materials/blob/main/img/boosting_0.PNG?raw=true'>

Похожей логки можно придерживаться при обучении других алгоритмов. 

**Интуиция за алгоритмом градиентного бустинга**: итеративно применять паттерны отклонений и улучшать предсказания. Как только мы достигли момента, когда отклонения не имеют никакого паттерна, мы прекращаем достраивать нашу модель. Общая последовательность действий такая:
- Сначала строим простые модели и анализируем ошибки
- Определяем точки, которые не вписываются в простую модель
- Добавляем модели, которые обрабатывают сложные случаи, которые были выявлены на начальной модели
- Собираем все построенные модели, определяя вес каждого предсказателя

Иллюстрация итеративного процесса представлена ниже:

<img src='https://github.com/MalikaL17/course_materials/blob/main/img/boosting_1.PNG?raw=true'>
В итоге, на 50 итерации получаем:
<img src='https://github.com/MalikaL17/course_materials/blob/main/img/boosting_2.PNG?raw=true'>

**Итоговое предсказание модели** - взвешенная сумма предсказаний слабых алгоритмов:
$$ prediction = \sum_{i=1}^m C_i\cdot a_i  $$ 
$C_i$ - весовые коэффициенты    
$a_i$ - предсказания слабых алгоритмов

### Градиентный спуск

Цель любого алгоритма обучения с учителем — определить функцию потерь и минимизировать её. Давайте обратимся к математике градиентного бустинга. Пусть, например, в качестве функции потерь будет среднеквадратичная ошибка (MSE):

$$ MSE = \sum(Y_i - Y_i^p)^{2} $$

Мы хотим построить наши предсказания таким образом, чтобы MSE была минимальна. Используя градиентный спуск и обновляя предсказания, основанные на скорости обучения (learning rate), ищем значения, на которых MSE минимальна.

Для объяснения метода градиентного бустинга полезно воспользоваться следующей аналогией. Бустинг можно представить как гольфиста, цель которого – загнать мяч в лунку с координатой yball. Положение мяча здесь – ответ композиции a(xball). Гольфист мог бы один раз ударить по мячу, не попасть в лунку и пойти домой, но настырность заставляет его продолжить

<img src="https://github.com/MalikaL17/course_materials/blob/main/img/gradient_decent.png?raw=true">

По счастью, ему не нужно начинать каждый раз с начальной позиции. Следующий удар гольфиста переводит мяч из текущего положения ak(xball) в положение ak+1(xball). Каждый следующий удар – это та поправка, которую вносит очередной базовый алгоритм в композицию. Если гольфист все делает правильно, то функция потерь будет уменьшаться, то есть мяч постепенно будет приближаться к лунке.

Удары при этом делаются не хаотично. Гольфист оценивает текущее положение мяча относительно лунки и следующим ударом старается нивелировать те проблемы, которые он создал себе всеми предыдущими. Подбираясь к лунке, он будет бить всё аккуратнее и, возможно, даже возьмет другую клюшку, но точно не будет лупить так же, как из первоначальной позиции. В итоге комбинация всех ударов рано или поздно перенесет мяч в лунку.

Подобно тому, как гольфист постепенно подводит мяч к цели, бустинг с каждым новым базовым алгоритмом всё больше приближает предсказание к истинному значению метки объекта.

В нашей задаче мы хотим найти направление, вдоль которого уменьшается функция потерь (MSE). Для этой цели нам нужно найти **антиградиент** функции потерь.
Из математики мы помним, что **градиент функции** - это вектор, указывающий на направление скорейшего роста функции. Тогда **антиградиент** - направление скорешего убывания функции. Полученное значение антиградиента используется для обучения следующего алгоритма, вместо обычной ошибки.  

### Скорость обучения (learning rate)

Обучение композиции с помощью градиентного бустинга может привести к переобучению, если базовые алгоритмы слишком сложные. Например, если сделать решающие деревья слишком глубокими (более 10 уровней), то при обучении бустинга ошибка на обучающей выборке даже при довольно скромном K может приблизиться к нулю, то есть предсказание будет почти идеальным, но на тестовой выборке всё будет плохо.

Существует два решения этой проблемы:
- Упростить базовую модель, уменьшив глубину дерева (либо примерив какие-либо ещё техники регуляризации)
- Ввести параметр, называемый скоростью (темпом) обучения - **learning rate**,  η ∈(0,1], который отвечает за то, чтобы каждый базовый алгоритм вносит относительно небольшой вклад в результат. Фактически **learning rate** представляет собой вес вклада базового алгоритма в итоговое решение

В результате, итоговое предсказание алгоритма будет выглядеть следующим образом:

$$ prediction = a_1 + \sum_{i=2}^m \eta \cdot C_i\cdot a_i  $$ 

$a_1$ - 1-й базовый алгоритм, который предсказывает исходную целевую переменную 

$a_i$ - предсказания i-x базовых алгоритмов, предсказывающих ошибку на i-1 шаге (2-й алгоритм и последующие)  
$C_i$ - весовые коэффициенты алгоритмов   
$\eta$ - скорость обучения  


Есть несколько реализаций градиентного бустинга:

1. Градиентный бустинг из sklearn
2. XGBoost
3. LightGBM
4. CatBoost
...

В рамках данного занятия рассмотрим градиентный бустинг из sklearn.

In [46]:
data_boston = load_boston()
df = pd.DataFrame(data_boston.data, columns=data_boston.feature_names)
df['Price'] = data_boston.target
df.head()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,Price
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33,36.2


**Дерево решений**

In [51]:
columns_to_use = ['CRIM', 'ZN', 'INDUS', 'CHAS', 'NOX', 'RM', 'AGE', 'DIS', 'RAD', 'TAX', 'PTRATIO', 'B', 'LSTAT']
X = df[columns_to_use]
y = df['Price']

tree = DecisionTreeRegressor()

kf = KFold(n_splits=5, shuffle=True, random_state=42)
scores = cross_val_score(tree, X, y, cv=kf, scoring='neg_mean_absolute_error') 
print("Значения точности при перекрестной проверке: \n{}".format(scores.mean()))

Значения точности при перекрестной проверке: 
-2.983275092215104


**Случайный лес**

In [56]:
from sklearn.ensemble import RandomForestRegressor

columns_to_use = ['CRIM', 'ZN', 'INDUS', 'CHAS', 'NOX', 'RM', 'AGE', 'DIS', 'RAD', 'TAX', 'PTRATIO', 'B', 'LSTAT']
X = df[columns_to_use]
y = df['Price']

rf = RandomForestRegressor()

kf = KFold(n_splits=5, shuffle=True, random_state=42)
scores = cross_val_score(rf, X, y, cv=kf, scoring='neg_mean_absolute_error') 
print("Значения точности при перекрестной проверке: \n{}".format(scores.mean()))

Значения точности при перекрестной проверке: 
-2.209312017084061


**Градиентный бустинг** 

Параметр **loss** (функция потерь) необходимо указывать в объекте моделе бустинга, т.к. именно по указанной функции потерь будет обучаться бустинг.

In [57]:
from sklearn.ensemble import GradientBoostingRegressor

gbr =  GradientBoostingRegressor(loss='lat', random_state=42)

kf = KFold(n_splits=5, shuffle=True, random_state=42)
scores = cross_val_score(rf, X, y, cv=kf, scoring='neg_mean_absolute_error') 
print("Значения точности при перекрестной проверке: \n{}".format(scores.mean()))

Значения точности при перекрестной проверке: 
-2.195887148126577


Видно, что при параметрах по умолчанию градиентный бустинг выдает наименьшую ошибку.